In [1]:
from pymongo import MongoClient
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import base64
import io

# Define the class names (ensure these are in the same order as during training)
class_names = ['organic', 'recycle'] 

# Load the trained model
model_path = 'model_small.h5'
model = load_model(model_path)

# MongoDB connection
client = MongoClient('mongodb://localhost:27017')
db = client['product_db']
collection = db['products']

def preprocess_image(img: Image.Image, img_width: int, img_height: int):
    """Preprocess the image for model prediction."""
    img = img.resize((img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

def predict_image(img_array):
    """Predict the class of the image."""
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_names[predicted_class_index]
    return predicted_class

def update_product_category(product_id, category):
    """Update the product category in MongoDB."""
    collection.update_one({"_id": product_id}, {"$set": {"category": category}})
    print(f"Updated product ID {product_id} with category '{category}'")

def process_and_update_products():
    """Retrieve products from MongoDB, process images, and update categories."""
    products = collection.find()
    
    for product in products:
        if 'image' in product:
            try:
                # Decode the base64 image
                img_data = base64.b64decode(product['image'])
                img = Image.open(io.BytesIO(img_data))
                img = img.convert("RGB")

                # Preprocess the image and make a prediction
                img_array = preprocess_image(img, img_width=80, img_height=45)  # Ensure these match your model's input size
                predicted_class = predict_image(img_array)
                
                # Update MongoDB with the predicted category
                update_product_category(product['_id'], predicted_class)

            except Exception as e:
                print(f"Error processing product ID {product['_id']}: {e}")

if __name__ == '__main__':
    process_and_update_products()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Updated product ID 66d44ba03472a6465732871b with category 'recycle'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Updated product ID 66d44d2cab31679d1b720c93 with category 'recycle'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Updated product ID 66d44d87182dcae8bfd4f670 with category 'recycle'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Updated product ID 66d452a79f18212201c84e48 with category 'organic'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Updated product ID 66d452c19f18212201c84e49 with category 'recycle'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Updated product ID 66d453d80e4422462f460679 with category 'organic'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Updated product ID 66d454fc0e4422462f46067a with category 'recycle'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Updated product ID 66d4555b0e4422462f46067b with category 'organic'


In [4]:
from pymongo import MongoClient
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import base64
import io

# Define the class names (ensure these are in the same order as during training)
class_names = ['organic', 'recycle']  # Replace with your actual class names

# Load the trained model
model_path = 'model_small.h5'
model = load_model(model_path)

# MongoDB connection
client = MongoClient('mongodb://localhost:27017')
db = client['product_db']
collection = db['products']

def preprocess_image(img: Image.Image, img_width: int, img_height: int):
    """Preprocess the image for model prediction."""
    img = img.resize((img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

def predict_image(img_array):
    """Predict the class of the image."""
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_names[predicted_class_index]
    return predicted_class

def update_product_category(product_id, category):
    """Update the product category in MongoDB."""
    result = collection.update_one({"_id": product_id}, {"$set": {"category": category}})
    if result.modified_count > 0:
        print(f"Product ID {product_id} updated with category '{category}'")
    else:
        print(f"Product ID {product_id} not updated")

def process_and_update_products():
    """Retrieve products from MongoDB, process images, and update categories."""
    products = collection.find()
    
    for product in products:
        if 'image' in product:
            try:
                # Decode the base64 image
                img_data = base64.b64decode(product['image'])
                img = Image.open(io.BytesIO(img_data))
                img = img.convert("RGB")

                # Preprocess the image and make a prediction
                img_array = preprocess_image(img, img_width=80, img_height=45)  # Ensure these match your model's input size
                predicted_class = predict_image(img_array)
                
                # Update MongoDB with the predicted category
                update_product_category(product['_id'], predicted_class)

            except Exception as e:
                print(f"Error processing product ID {product['_id']}: {e}")

if __name__ == '__main__':
    process_and_update_products()


In [2]:
from pymongo import MongoClient
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import base64
import io
from bson import ObjectId  # To handle MongoDB ObjectId

# Define the class names (ensure these are in the same order as during training)
class_names = ['organic', 'recycle']  # Replace with your actual class names

# Load the trained model
model_path = 'model_small.h5'  # Ensure this path is correct relative to your script
model = load_model(model_path)

# MongoDB connection
client = MongoClient('mongodb://localhost:27017')
db = client['ecommerce_db']  # Updated database name
collection = db['products']

# Image preprocessing parameters (ensure these match your model's input size)
IMG_WIDTH = 80
IMG_HEIGHT = 45

def preprocess_image(img: Image.Image, img_width: int, img_height: int):
    """
    Preprocess the image for model prediction.
    
    Args:
        img (Image.Image): PIL Image.
        img_width (int): Width to resize.
        img_height (int): Height to resize.
    
    Returns:
        np.ndarray: Preprocessed image array.
    """
    img = img.resize((img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize to [0,1]
    return img_array

def predict_image(img_array):
    """
    Predict the class of the image.
    
    Args:
        img_array (np.ndarray): Preprocessed image array.
    
    Returns:
        str: Predicted class name.
    """
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_names[predicted_class_index]
    return predicted_class

def update_product_category(product_id, category):
    """
    Update the product category in MongoDB.
    
    Args:
        product_id (ObjectId): The MongoDB ObjectId of the product.
        category (str): The predicted category.
    """
    result = collection.update_one(
        {"_id": ObjectId(product_id)},
        {"$set": {"category": category}}
    )
    if result.modified_count > 0:
        print(f"Product ID {product_id} updated with category '{category}'")
    else:
        print(f"Product ID {product_id} not updated or already has category '{category}'")

def process_and_update_products():
    """
    Retrieve products from MongoDB, process images, and update categories.
    """
    # Fetch all products that have an image and category is default or undefined
    query = {
        "image": {"$exists": True, "$ne": None},
        "category": {"$in": ["under processing category"]}  # Adjust based on your default category
    }
    products = collection.find(query)
    
    for product in products:
        product_id = product['_id']
        try:
            # Decode the base64 image
            img_data = base64.b64decode(product['image'])
            img = Image.open(io.BytesIO(img_data))
            img = img.convert("RGB")
    
            # Preprocess the image and make a prediction
            img_array = preprocess_image(img, IMG_WIDTH, IMG_HEIGHT)
            predicted_class = predict_image(img_array)
            
            # Update MongoDB with the predicted category
            update_product_category(product_id, predicted_class)
    
        except Exception as e:
            print(f"Error processing product ID {product_id}: {e}")

if __name__ == '__main__':
    process_and_update_products()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Product ID 671d1e8187cf34cb41f3aa42 updated with category 'recycle'
